In [1]:
%%capture
!pip install transformers

from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 890kB 2.5MB/s 
     |████████████████████████████████| 1.1MB 14.6MB/s 
     |████████████████████████████████| 3.0MB 20.6MB/s 
     |████████████████████████████████| 890kB 33.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=a5b9cd7809f2fbcb8032b9bc3777df3fbada482a515c92dfaa62a65aded65841
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
#!wget --directory-prefix='/content/drive/My Drive/colab_files/word_embeddings/' http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
#!wget --directory-prefix='/content/drive/My Drive/colab_files/InferSent/encoder/' https://dl.fbaipublicfiles.com/infersent/infersent1.pkl

In [ ]:
#import zipfile
#with zipfile.ZipFile('/content/drive/My Drive/colab_files/word_embeddings/glove.6B.zip', 'r') as zip_ref:
#    zip_ref.extractall('/content/drive/My Drive/colab_files/word_embeddings/glove/')

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/colab_files/modules')

import infersent_glove_context_generation as ig

import time
import os
import contextlib
import torch
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
def extract_answer_phrase(question, context):
    '''
    Takes a `question` string and an `context` string (which contains the
    answer), and identifies the words within the `context` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, context)

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    #print('Answer: "' + answer + '"')
    return answer

In [ ]:
def find_answer(url, question):
    context = ig.generate_context(url, question)
    answer = extract_answer_phrase(question, context)
    return answer

**Inference:**

In [ ]:
url = 'https://en.wikipedia.org/wiki/India'
question = 'Which sports does India play?'

start_time = time.time()
with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    answer = find_answer(url, question)
end_time = time.time()
print('Answer: ', answer)
print('\n\nExecution Time: {} seconds'.format(end_time - start_time))

Answer:  cricket is the most popular sport in india . in india , several traditional indigenous sports remain fairly popular , such as kabaddi , kho kho , pehlwani and gilli - danda . india has traditionally been the dominant country at the south asian games . corruption in india is perceived to have decreased . other sports in which indians have succeeded internationally include badminton ( saina nehwal and p v sindhu are two of the top - ranked female badminton players in the world ) , boxing , and wrestling


Execution Time: 50.703139781951904 seconds


In [ ]:
url = 'https://en.wikipedia.org/wiki/India'
question = 'What name did the Greeks use to refer to Indians?'

start_time = time.time()
with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    answer = find_answer(url, question)
end_time = time.time()
print('Answer: ', answer)
print('\n\nExecution Time: {} seconds'.format(end_time - start_time))

Answer:  indoi


Execution Time: 30.312913179397583 seconds


In [ ]:
url = 'https://en.wikipedia.org/wiki/India'
question = 'Which are the major rivers in India'

start_time = time.time()
with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
    answer = find_answer(url, question)
end_time = time.time()
print('Answer: ', answer)
print('\n\nExecution Time: {} seconds'.format(end_time - start_time))

Answer:  ganges and the brahmaputra


Execution Time: 29.730848789215088 seconds


**FLASK**

In [ ]:
!pip install flask-ngrok

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request

app = Flask(__name__, template_folder='drive/My Drive/colab_files/templates')
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route("/")
def home():
    return render_template('home.html')

@app.route('/about')
def about():
    return render_template('about.html')

@app.route('/handle_data', methods=['GET', 'POST'])
def handle_data():
    url = request.form['inputURL']
    question = request.form['inputQuestion']
    answer = find_answer(url, question)

    return render_template('handle_data.html', result_url=url, result_question=question,
                           result_answer=answer)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6bdd333bf7be.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [15/Sep/2020 10:04:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Sep/2020 10:04:20] "GET /static/css/template.css HTTP/1.1" 404 -
127.0.0.1 - - [15/Sep/2020 10:04:20] "GET /favicon.ico HTTP/1.1" 404 -
